In [19]:
from llm.llm.chatgpt import ChatGPT
from llm.llm.gemini import Gemini
from setup_db import create_chroma_db, connect_to_db, execute_query, DBHUB
from branch_reasoning import *

In [20]:
llm = Gemini()

In [21]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [22]:
conn = connect_to_db(db_name, user, password, host, port)

Connecting to database test_db, postgres...


In [23]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'company_name_chroma'
persist_directory = 'data/company_name_chroma'
vector_db_company = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'sql_query'
persist_directory = 'data/sql_query'
vector_db_sql = create_chroma_db(collection_chromadb, persist_directory)

In [24]:
vector_db_company.similarity_search('company', k=5)

[Document(metadata={'lang': 'en', 'stock_code': 'SSI'}, page_content='SSI Securities Corporation'),
 Document(metadata={'lang': 'vi', 'stock_code': 'SSI'}, page_content='SSI'),
 Document(metadata={'lang': 'en', 'stock_code': 'MSN'}, page_content='Masan Group Corporation'),
 Document(metadata={'lang': 'vi', 'stock_code': 'VIC'}, page_content='VIC'),
 Document(metadata={'lang': 'en', 'stock_code': 'SSI'}, page_content='SSI Securities')]

In [25]:
db = DBHUB(conn, bank_vector_store, none_bank_vector_store, vector_db_company, vector_db_sql)

In [26]:
prompt =  "Calculate ROA, ROE of all the company which are owned by VinGroup"

In [27]:
history = llm_branch_reasoning(llm, prompt, db, verbose=True)

prompt_token_count: 379
candidates_token_count: 206
total_token_count: 585

Branch reasoning response: 
```json
{
  "steps": [
    "Step 1: Identify all companies owned by VinGroup using the `company_info` and `sub_and_shareholder` databases. This involves joining the tables based on relevant identifiers (e.g., company IDs) to filter for companies where VinGroup holds ownership.",
    "Step 2: Retrieve the necessary financial data (Net Income, Total Assets, Total Equity) for the identified VinGroup-owned companies from either the `bank_financial_report` or `non_bank_financial_report` database, depending on the company type.  Handle potential missing data appropriately (e.g., imputation or exclusion).",
    "Step 3: Calculate ROA (Net Income / Total Assets) and ROE (Net Income / Total Equity) for each company identified in Step 1 using the financial data retrieved in Step 2.  Ensure consistent accounting standards are applied across all calculations."
  ]
}
```

prompt_token_count: 387


AttributeError: 'NoneType' object has no attribute 'to_markdown'